**uniForest *v*.1**
---


**Citation:**

Leigh R.J., Murphy R.A., and Walsh F. (2021) Paper title, Journal metrics


**Description:**

uniForest is a user-friendly script for outlier processing in microbiome studies 

**Usage**

**Please read the manual**

---

Each executable cell can be ran using the black mouseover to the left of their titles (denoted in **bold**). Simply upload a file, set your desired parameters, and run cells. Some cells display a lot of messages. These can be hidden after they're produced using the black mouseover in their output cell. 

After running the "Data processing" cell, an imputed dataset, change log, and descriptive statistics table are available to download using the **folder icon on the left side of the screen.**


The code underlying each cell can be inspected by double clicking a given cell.

In [ ]:
#@title **Library installation**
#@markdown This process installs specific Python library versions required to run this script. Several messages will be produced.

!pip install scikit-learn==0.24 scikit-bio

In [3]:
#@title **Import Python modules for data processing**
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn import preprocessing
from skbio.stats.distance import anosim, permanova
from skbio.diversity import beta_diversity as beta_div
#from google.colab import files
import io
import itertools

In [4]:
#@title **File upload**
#@markdown A single tsv file is required for upload. This file must have taxa in a single column on the left and a single row on top labelled "Groups" (in cell A1). The "Groups" row specifies what group a given sample belongs to (eg. Lung_pretreatment or Soil_arctic). All samples of a given group should be beside each other.
from google.colab import files
dataset = files.upload()

Saving human3.tsv to human3 (1).tsv


In [5]:
#@title **Metric specifications**
#@markdown This cell defines the impution metric, whether data is to be scaled, whether Isolation Forest (iForest) bootstrapping is to be performed and the stringency of iForest.

#@markdown Data **scaling** ensures all samples have an equivalent sum. This is to mitigate errors due to data with low coverage/depth.

#@markdown iForest **bootstrapping** can be turned on with **"True**". Default usage is **"False"**.

#@markdown iForest stringency is set using **contamination**. This value can be set between 0 (least stringent) and 0.5 (most stringent). Numerical values are set **without quotation marks**. The iForest algorithm can set this value automatically for each sample using **"auto"**. The default is **"auto"** (**with quotation marks**).

#@markdown 

scale_data = "yes"
iForest_contamination="auto"
iForest_bootstrap="False"

In [ ]:
#@title **Data processing**
datasetName = next(iter(dataset))
dataset_df = pd.read_csv(io.BytesIO(dataset[datasetName]), sep='\t', header=None).set_index(0)
GroupColours = dataset_df.iloc[0].tolist()

t = set(GroupColours)
c = sorted(set(itertools.combinations(t, 2)))

GroupedData = dataset_df.T.groupby("Group", axis=0)
Groups = set(GroupColours)
#Metric = metric


def Metric(group):
  if metric == "mean":
    Metric = np.mean(group)
  if metric == "median":
    Metric = np.median(group)
  if metric == "geometric":
    Metric = float(stats.gmean(group).item(0))
  if metric == "harmonic":
     Metric=float(stats.hmean(group).item(0))
  return Metric

def PairwiseDescriptiveStats(Group1, Group2):
  L, S = stats.levene(Group1, Group2)
  results = [np.mean(Group1), np.std(Group1), np.var(Group1), np.median(Group1), stats.hmean(Group1).item(0), stats.gmean(Group1).item(0), min(Group1), max(Group1), np.mean(Group2), np.std(Group2), np.var(Group2), np.median(Group2), stats.hmean(Group2).item(0), stats.gmean(Group2).item(0), min(Group2), max(Group2), L, S]
  return results

def Scale(item):
    scaler = item.astype(float).sum(axis=0)/item.astype(float).sum(axis=0).max()
    scaledData = item.astype(float)/scaler
    scaledInputData = scaledData.values.tolist()
    return scaledData, scaledInputData


def iForest():
  import warnings
  warnings.filterwarnings('ignore') #This turns off the annoying numpy and scipy warnings
  imputedDataset = pd.DataFrame()
  changeLog = []

  '''
  The next section performs iForest and imputes outliers
  '''

  for Group in Groups:
    List_Imputed = []
    rawDataset = GroupedData.get_group(Group).set_index("Group")
    outliersRemoved_list = []
    imputedDataset_list = []

    for taxonID, dataSeries in rawDataset.iteritems():
      imputedSeries = []
      rawSeries = np.array(pd.Series(dataSeries).astype(np.float)).reshape(-1,1)
      clf = IsolationForest(contamination=iForest_contamination, bootstrap=iForest_bootstrap, random_state=42)
      preds = clf.fit_predict(rawSeries)
      outliersRemovedSeries = rawSeries[np.where(preds == 1, True, False)]
      if len(outliersRemovedSeries) != 0:
        imputer = float(Metric(outliersRemovedSeries))
        for item in rawSeries:
          if item in outliersRemovedSeries:
            imputedSeries.append(float(item))
          else:
            imputedSeries.append(imputer)
      else:
        imputedSeries = list(np.concatenate(rawSeries).flat)
      imputedDataset_list.append(imputedSeries)
    #List_imputed = pd.DataFrame.from_records(np.array(imputedDataset_list).T, index=rawDataset.index, columns=dataset_df.index.drop("Group"))
    #imputedDataset = imputedDataset.append(List_imputed) #.reindex(List_imputed.index)
     
      '''
      The next section computes statistics for raw data, inlier data, and imputed data

      '''
     
      raw_descriptiveStatistics = [np.mean(rawSeries), np.std(rawSeries), 
                                   np.median(rawSeries), np.var(rawSeries), 
                                   stats.hmean(np.float64(rawSeries)).item(0), stats.gmean(np.float64(rawSeries)).item(0), 
                                   stats.variation(rawSeries).item(0), 
                                   min(rawSeries), max(rawSeries)]
      inlier_descriptiveStatistics = [np.mean(outliersRemovedSeries), np.std(outliersRemovedSeries), 
                                      np.median(outliersRemovedSeries), np.var(outliersRemovedSeries), 
                                      stats.hmean(np.float64(outliersRemovedSeries)).item(0), stats.gmean(np.float64(outliersRemovedSeries)).item(0), 
                                      stats.variation(outliersRemovedSeries).item(0), 
                                      min(rawSeries), max(rawSeries)]
      imputed_descriptiveStatistics = [np.mean(imputedSeries), np.std(imputedSeries), 
                                       np.median(imputedSeries), np.var(imputedSeries), 
                                       stats.hmean(np.float64(imputedSeries)).item(0), stats.gmean(np.float64(imputedSeries)).item(0), 
                                       stats.variation(imputedSeries).item(0), 
                                       min(rawSeries), max(rawSeries)]

      raw_descriptiveStatistics = list(np.float_(raw_descriptiveStatistics))
      inlier_descriptiveStatistics = list(np.float_(inlier_descriptiveStatistics))
      imputed_descriptiveStatistics = list(np.float_(imputed_descriptiveStatistics))
      Levenes_W, Levenes_P = stats.levene(np.concatenate(rawSeries).flat, imputedSeries)

      numberInliers = len(outliersRemovedSeries)
      numberDataset = len(rawSeries)
      numberOutliers = numberDataset-numberInliers
      proportionInliers = numberOutliers/numberDataset
      varianceReduction = 100-((np.var(imputedSeries)/np.var(rawSeries)*100))
      CoVreduction = 100-((stats.variation(imputedSeries)/stats.variation(rawSeries)*100))
      
      descriptiveStatistics = [Group, taxonID, numberDataset, numberInliers, numberOutliers, proportionInliers] + raw_descriptiveStatistics + inlier_descriptiveStatistics + imputed_descriptiveStatistics + [float(varianceReduction), float(CoVreduction), float(Levenes_W), float(Levenes_P)]
      
      changeLog.append(descriptiveStatistics)

    List_imputed = pd.DataFrame.from_records(np.array(imputedDataset_list).T, index=rawDataset.index, columns=dataset_df.index.drop("Group"))  
    imputedDataset = imputedDataset.append(List_imputed)

  transposed_imputedDataset = imputedDataset.sort_index().T
  grouped_imputedDataset = imputedDataset.groupby("Group",axis=0)
  changeStatistics = pd.DataFrame.from_records(changeLog, columns = ["GroupID", "taxonID", "DatasetSize", "InliersSize", "OutliersSize", "OutliersProprotion", "Mean", "StandardDeviation", "Median", "Variance", "HarmonicMean", "GeometricMean", "Coefficient_of_Variation", "Minimum", "Maximum", "Mean", "StandardDeviation", "Median", "Variance", "HarmonicMean", "GeometricMean", "Coefficient_of_Variation", "Minimum", "Maximum", "Mean", "StandardDeviation", "Median", "Variance", "HarmonicMean", "GeometricMean", "Coefficient_of_Variation", "Minimum", "Maximum", "VarianceReduction", "CoV_Reduction", "Levenes_W", "Levenes_P"])
  return changeStatistics, imputedDataset, transposed_imputedDataset, grouped_imputedDataset



def Components(item, figHeight, figWidth):
  df = pd.DataFrame.from_records(item) #.drop(0, axis=1)
  pca = PCA()
  scaled_data = preprocessing.scale(df.T)
  components = pca.fit_transform(scaled_data)
  per_var = np.round(pca.explained_variance_ratio_* 100, decimals=2)
  fig = px.scatter_3d(
      components, x=0, y=1, z=2, color=Colours,
      labels={'0': 'PC1 - {0}%'.format(per_var[0]), 
              '1': 'PC2 - {0}%'.format(per_var[1]), 
              '2': 'PC3 - {0}%'.format(per_var[2])}
    )

  fig.update_layout(margin=dict(l=1, r=1, b=1, t=1), autosize=True, width=figWidth,height=figHeight)
  fig.update_layout(legend=dict(
    yanchor="top",
     y=0.99,
    xanchor="left",
    x=0.01))
  
  fig.show()
  return   

def BetaDiversity(Data, GroupIDs):
  bc = beta_div("braycurtis", Data, validate=True)
  anosim_result = anosim(bc, GroupIDs, permutations=0)
  stat, pvalue = anosim_result['test statistic'], anosim_result['p-value']
  return bc, stat, pvalue

def separationDistancesBetweenGroups(GroupedData, ImputedGroupData):
  Results = []
  t = set(GroupColours)
  c = sorted(set(itertools.combinations(t, 2)))
  for item in c:
    raw_GroupA = GroupedData.get_group(item[0]).set_index("Group")
    raw_GroupB = GroupedData.get_group(item[1]).set_index("Group")
    raw_Groups = pd.concat([raw_GroupA, raw_GroupB])
    imputed_GroupA = ImputedGroupData.get_group(item[0])
    imputed_GroupB = ImputedGroupData.get_group(item[1]) 
    imputed_Groups = pd.concat([imputed_GroupA, imputed_GroupB]) 
    IDs = list(raw_Groups.index)
    x1,y1,z1 = BetaDiversity(raw_Groups, IDs)
    x2,y2,z2 = BetaDiversity(imputed_Groups, IDs)
    y3, y4 = (y1+1)/2, (y2+1)/2
    x3 = [metricID, item[0], item[1], y3, y4, (y4/y3)-1]
    Results.append(x3)
  results_df = pd.DataFrame.from_records(Results, columns = ["Metric", "GroupA", "GroupB", "rawScore", "processedScore", "Improvement"])
  return results_df
  #Results.to_csv("Improvements.tsv", sep='\t', index=False)


def separationDistancesWithinGroups(GroupedData, ImputedGroupData):
  distances =[]
  for item in t:
    raw_Group = GroupedData.get_group(item).set_index("Group")
    imputed_Group = ImputedGroupData.get_group(item) #.set_index("Group")
    pca = PCA()
    f = []
    g = []
    scaled_data = preprocessing.scale(raw_Group)
    components = pca.fit_transform(scaled_data)
    a1 = pca.components_[0:3].T
    b1 = pca.explained_variance_ratio_[0:3]
    c1 = a1*b1
    d1 = itertools.combinations(c1, 2)
    for m in d1:
      e1 = np.sqrt(np.sum((m[0]-m[1])**2, axis=0))
      f.append(e1)
    scaled_data = preprocessing.scale(imputed_Group)
    components = pca.fit_transform(scaled_data)
    a2 = pca.components_[0:3].T
    b2 = pca.explained_variance_ratio_[0:3]
    c2 = a2*b2
    d2 = itertools.combinations(c2, 2)
    for m in d2:
      e2 = np.sqrt(np.sum((m[0]-m[1])**2, axis=0))
      g.append(e2)
    L, S = stats.levene(f, g)
    statistics = [item, np.mean(f), np.std(f), np.var(f), np.median(f), stats.hmean(f).item(0), stats.gmean(f).item(0), min(f), max(f), np.mean(g), np.std(g), np.var(g), np.median(g), stats.hmean(g).item(0), stats.gmean(g).item(0), min(g), max(g), L, S]
    distances.append(statistics)
  distances_df = pd.DataFrame(distances, columns = ["Group", "Mean", "StandardDeviation", "Variance", "Median", "HarmonicMean", "GeometricMean", "minimum", "maximum", "Mean", "StandardDeviation", "Variation", "Median", "HarmonicMean", "GeometricMean", "minimum", "maximum", "Levenes_W", "Levenes_P"])
  return distances_df
#df.to_csv("Distances.tsv", sep='\t', header=True, index=False)


from skbio.diversity import alpha as alpha_div

def AlphaDiversityStatistics(rawDataset, imputedDataset):
  #dataset_df1 = pd.DataFrame(columns=rawDataset.iloc[0])
  dataset_df1 = rawDataset[1:]
  dataset_df1.columns = rawDataset.iloc[0]
  raw_Chao1 = dataset_df1.astype(float).apply(alpha_div.chao1, axis=0).apply(pd.to_numeric).sort_index()
  raw_Shannon = dataset_df1.astype(float).apply(alpha_div.shannon, axis=0).apply(pd.to_numeric).sort_index()
  raw_SimpsonD = dataset_df1.astype(float).apply(alpha_div.simpson, axis=0).apply(pd.to_numeric).sort_index()
  raw_SimpsonE = dataset_df1.astype(float).apply(alpha_div.simpson_e, axis=0).apply(pd.to_numeric).sort_index()
  imputed_Chao1 = imputedDataset.astype(float).apply(alpha_div.chao1, axis=0).apply(pd.to_numeric).sort_index()
  imputed_Shannon = imputedDataset.astype(float).apply(alpha_div.shannon, axis=0).apply(pd.to_numeric).sort_index()
  imputed_SimpsonD = imputedDataset.astype(float).apply(alpha_div.simpson, axis=0).apply(pd.to_numeric).sort_index()
  imputed_SimpsonE = imputedDataset.astype(float).apply(alpha_div.simpson_e, axis=0).apply(pd.to_numeric).sort_index()
  AlphaDObservations = [raw_Chao1, raw_Shannon, raw_SimpsonD, raw_SimpsonE, imputed_Chao1, imputed_Shannon, imputed_SimpsonD, imputed_SimpsonE]
  AlphaDObservations = pd.concat(AlphaDObservations, axis=1).rename(columns={0:"Chao1_raw", 1:"Shannon_raw", 2:"SimpsonsD_raw", 3:"SimpsonsE_raw", 4:"Chao1_processed", 5:"Shannon_processed", 6:"SimpsonsD_processed", 7:"SimpsonsE_processed"})
  AlphaDGroups = AlphaDObservations.groupby("Group")
  AlphaGroupStats = []    
  for Group in Groups:
    AlphaGroup = AlphaDGroups.get_group(Group)
    raw_Chao1_group = AlphaGroup["Chao1_raw"] 
    raw_Shannon_group = AlphaGroup["Shannon_raw"]
    raw_SimpsonsD_group = AlphaGroup["SimpsonsD_raw"]  
    raw_SimpsonsE_group = AlphaGroup["SimpsonsE_raw"]
    imputed_Chao1_group = AlphaGroup["Chao1_processed"] 
    imputed_Shannon_group = AlphaGroup["Shannon_processed"]
    imputed_SimpsonsD_group = AlphaGroup["SimpsonsD_processed"]  
    imputed_SimpsonsE_group = AlphaGroup["SimpsonsE_processed"]
    results = PairwiseDescriptiveStats(raw_Chao1_group, imputed_Chao1_group)
    results = [Group, "Chao1"] + results
    AlphaGroupStats.append(results)
    results = PairwiseDescriptiveStats(raw_Shannon_group, imputed_Shannon_group)
    results = [Group, "Shannon"] + results
    AlphaGroupStats.append(results)
    results = PairwiseDescriptiveStats(raw_SimpsonsD_group, imputed_SimpsonsD_group)
    results = [Group, "SimpsonsD"] + results
    AlphaGroupStats.append(results)
    results = PairwiseDescriptiveStats(raw_SimpsonsE_group, imputed_SimpsonsE_group)
    results = [Group, "SimpsonsE"] + results
    AlphaGroupStats.append(results)
  AlphaGroupStats = pd.DataFrame(AlphaGroupStats, columns = ["Group", "Metric", "Mean", "StandardDeviation", "Variance", "Median", "HarmonicMean", "GeometricMean", "minimum", "maximum", "Mean", "StandardDeviation", "Variation", "Median", "HarmonicMean", "GeometricMean", "minimum", "maximum", "Levenes_W", "Levenes_P"])
  return AlphaDObservations, AlphaGroupStats

metric="optimal"

if metric=="optimal":
  Scores = []
  Metric_list=["mean","median","harmonic","geometric"]
  for new_metric in Metric_list:
    metricID = str(new_metric)
    #metric == new_metric
    if new_metric == "mean":
      Metric = np.mean
    if new_metric == "median":
      Metric = np.median
    if new_metric == "geometric":
      Metric = stats.gmean
    if new_metric == "harmonic":
       Metric=stats.hmean
    changeStats, imputed_df, transposed_imputed_df, grouped_imputed_df = iForest()
    interGroup_separation = separationDistancesBetweenGroups(GroupedData, grouped_imputed_df)
    intraGroup_separation = separationDistancesWithinGroups(GroupedData, grouped_imputed_df)
    alphaDiversityObservations, alphaDiversityStatistics = AlphaDiversityStatistics(dataset_df, transposed_imputed_df)
    changeStats.to_csv(metricID+"_changeLog.tsv", sep='\t', header=True, index=False)
    imputed_df.to_csv(metricID+"_imputedDataset.tsv", sep='\t', header=True, index=True)
    transposed_imputed_df.to_csv(metricID+"_imputedDataset2.tsv", sep='\t', header=True, index=True)
    interGroup_separation.to_csv(metricID+"_interGroup_separation.tsv", sep='\t', header=True, index=False)
    intraGroup_separation.to_csv(metricID+"_intraGroup_separation.tsv", sep='\t', header=True, index=False)
    alphaDiversityObservations.to_csv(metricID+"_alphaDiversityObservations.tsv", sep='\t', header=True, index=True)
    alphaDiversityStatistics.to_csv(metricID+"_alphaDiversityStatistics.tsv", sep='\t', header=True, index=True)
    #x = [metricID, stats.hmean(np.float64(interGroup_separation['Improvement'])).item(0)]
    x = [metricID, stats.hmean(interGroup_separation['rawScore']).item(0), stats.hmean(interGroup_separation['processedScore']).item(0), (stats.hmean(interGroup_separation['processedScore']).item(0)/stats.hmean(interGroup_separation['rawScore']).item(0))-1, np.median(interGroup_separation['rawScore']), np.median(interGroup_separation['processedScore']), (np.median(interGroup_separation['processedScore'])/np.median(interGroup_separation['rawScore'])-1), np.mean(interGroup_separation['rawScore']), np.mean(interGroup_separation['processedScore']), np.mean(interGroup_separation['processedScore'])/np.mean(interGroup_separation['rawScore'])-1]
    Scores.append(x)
  Score_df = pd.DataFrame.from_records(Scores, columns = ["Metric", "HarmonicRaw", "HarmonicProcessed", "HarmonicImprovement", "MedianRaw", "MedianProcessed","MedianImprovement", "MeanRaw", "MeanProcessed", "MeanImprovement"])
  Score_df.to_csv("Scores.tsv", sep='\t', header=True, index=True)
  print("The optimum metric is: "+Score_df[Score_df.MeanImprovement == Score_df.MeanImprovement.max()]['Metric'][1])
  optimal_viewing_metric = Score_df[Score_df.MeanImprovement == Score_df.MeanImprovement.max()]['Metric'][1]

Score_df

In [ ]:
#@title **Click here to change PCA settings**
#@markdown The impution metric can be changed so differences can be viewed here. This metric can be any of **"mean"**, **"median"**, **"geometric"** (geometric mean), or **"harmonic"** (harmonic mean). By default, this is set to the optimum metric. The graph width and lengths can be changed using the "graphWidth" and "graphHeight" parameters.
viewing_metric = optimal_viewing_metric
graphWidth = 1000
graphHeight = 1250

In [ ]:
#@title **View PCA (original data)**
#@markdown These graphs are fully interactive, they can be rotated and zoomed into, groups can also be hidden by clicking on their legend.
InputData = dataset_df.T.set_index("Group").sort_index().T
Colours = InputData.columns.to_list()

Components(InputData, graphWidth, graphHeight)

In [ ]:
#@title **View PCA (processed data)**
#@markdown These graphs are fully interactive, they can be rotated and zoomed into, groups can also be hidden by clicking on their legend.
if viewing_metric == "mean":
  ImputedData = pd.read_csv('/content/mean_imputedDataset.tsv', sep='\t', index_col='Group').astype(np.float64).sort_index().T
if viewing_metric == "median":
  ImputedData = pd.read_csv('/content/median_imputedDataset.tsv', sep='\t', index_col='Group').astype(np.float64).sort_index().T
if viewing_metric == "geometric":
  ImputedData = pd.read_csv('/content/geometric_imputedDataset.tsv', sep='\t', index_col='Group').astype(np.float64).sort_index().T
if viewing_metric == "harmonic":
  ImputedData = pd.read_csv('/content/harmonic_imputedDataset.tsv', sep='\t', index_col='Group').astype(np.float64).sort_index().T

Colours = ImputedData.columns.to_list()

Components(ImputedData, graphWidth, graphHeight)